In [1]:
import optuna
import optuna.visualization as vis
import numpy as np
from scipy.integrate import solve_ivp
from scipy.optimize import minimize

# MPC FUNCTIONS

In [2]:
# ========================
# MODEL DEFINITIONS
# ========================

def assumed_model(t, y, F):
    """Controller's simplified model"""
    X, S, V = y
    S = np.maximum(S, 0)  # Prevent negative substrate
    
    # Simplified growth model (no inhibition/decay)
    mu = mu_max * S / (Ks + S)
    dX_dt = mu * X - (F/V) * X
    dS_dt = -(1/Yxs) * mu * X + (F/V)*(Sin - S)
    dV_dt = F
    
    return np.array([dX_dt, dS_dt, dV_dt])

def actual_model(t, y, F):
    """Real system dynamics"""
    X, S, V = y
    S = max(S,0)  # Prevent negative substrate 
    # True growth model with inhibition and decay
    mu = mu_max * S / (Ks + S + S**2/K_I)
    dX_dt = mu * X - (F/V) * X - K_D * X
    dS_dt = -(1/Yxs) * mu * X + (F/V)*(Sin - S)
    dV_dt = F
    
    return np.array([dX_dt, dS_dt, dV_dt])

# ========================
# DISCRETIZED FUNCTIONS 
# ========================
def RK_MODEL(t_start, X, S, V, F, h, dt):
    """RK4 integration with time awareness"""
    Num = int(dt/h)
    t = t_start
    X_curr, S_curr, V_curr = X, S, V
    
    for _ in range(Num):
        k1 = assumed_model(t, [X_curr, S_curr, V_curr], F)
        k2 = assumed_model(t + h/2, [
            X_curr + k1[0]*h/2,
            S_curr + k1[1]*h/2,
            V_curr + k1[2]*h/2
        ], F)
        k3 = assumed_model(t + h/2, [
            X_curr + k2[0]*h/2,
            S_curr + k2[1]*h/2,
            V_curr + k2[2]*h/2
        ], F)
        k4 = assumed_model(t + h, [
            X_curr + k3[0]*h,
            S_curr + k3[1]*h,
            V_curr + k3[2]*h
        ], F)

        X_next = X_curr + (h/6)*(k1[0] + 2*k2[0] + 2*k3[0] + k4[0])
        S_next = S_curr + (h/6)*(k1[1] + 2*k2[1] + 2*k3[1] + k4[1])
        V_next = V_curr + (h/6)*(k1[2] + 2*k2[2] + 2*k3[2] + k4[2])
        
        X_curr, S_curr, V_curr = X_next, S_next, V_next
        t += h
        
    return X_curr, S_curr, V_curr

# ========================
# SETPOINT FUNCTION
# ========================
def Cb_set(t):
    return 30  # Constant setpoint

# ========================
# COST FUNCTION
# ========================
def cost_function(F_opt, X0, S0, V0, current_time, Q, R, Np, h, dt, method="RK"):
    """
    MPC cost function: evaluates tracking and control effort over prediction horizon.
    
    Parameters:
        F_opt: Array of feed rates (Np,)
        X0, S0, V0: Initial conditions
        current_time: Current simulation time
        Q, R: Weights for tracking error and control smoothness
        Np: Prediction horizon
        h: Integration step size
        dt: Time between control updates
        method: Integration method ("RK" or "Euler")

    Returns:
        J: Scalar cost value
    """
    J = 0
    X_curr, S_curr, V_curr = X0, S0, V0

    for k in range(Np):
        pred_time = current_time + k * dt
        setpoint = Cb_set(pred_time)

        # Predict next state
        if method == "Euler":
            X_next, S_next, V_next = Euler(X_curr, S_curr, V_curr, F_opt[k], h, dt)
        else:
            X_next, S_next, V_next = RK_MODEL(pred_time, X_curr, S_curr, V_curr, F_opt[k], h, dt)

        # Tracking error
        tracking_error = setpoint - X_next
        J += Q * tracking_error**2

        # Control effort smoothness
        if k > 0:
            delta_F = F_opt[k] - F_opt[k - 1]
            J += R * delta_F**2

        # Update states
        X_curr, S_curr, V_curr = X_next, S_next, V_next

    return J

In [3]:
# ========================
# CONSTARINT FUNCTIONS
# ========================

def Volume_constraint(F_opt, X, S, V, current_time):
    X_curr, S_curr, V_curr = X, S, V
    Volume_Log= []
    for k in range(Np):
        X_next, S_next, V_next = RK_MODEL(
            current_time + k*dt, X_curr, S_curr, V_curr, F_opt[k], h, dt
        )
        Volume_Log.append(V_next)
        X_curr, S_curr, V_curr = X_next, S_next, V_next
    
    return Vmax- max(Volume_Log)

def Substrate_constraint(F_opt, X, S, V, current_time):
    X_curr, S_curr, V_curr = X, S, V
    Substrate_Log= []
    for k in range(Np):
        X_next, S_next, V_next = RK_MODEL(
            current_time + k*dt, X_curr, S_curr, V_curr, F_opt[k], h, dt
        )
        Substrate_Log.append(S_next)
        X_curr, S_curr, V_curr = X_next, S_next, V_next
    
    return min(Substrate_Log) - Smin

        

In [4]:
# ========================
# SYSTEM PARAMETERS
# ========================
mu_max = 0.8564       # 1/h
Ks = 0.1707           # g/L
Yxs = 0.4066        # g/g
Sin = 286.0           # g/L
V0 = 1.6              # L, initial volume
X0 = 5              # g/L, initial biomass
S0 = 0.37            # g/L, initial substrate
F0 = 0.1              # L/h
K_I = 33              # Inhibition constant
K_D = 0.03            # Decay coefficient


# ========================
# MPC PARAMETERS
# ========================
dt = 0.1              # Control interval time (h)
Q =  3310          # State tracking weight
R =  151         # Control effort weight
h = 0.01              # integration step size
At = 6                # Total simulation time (h)
Np = 15            # Prediction horizon steps

#============================
# BOUNDS AND CONSTRAINTS
#============================

bnds = [(0,0.1) for _ in range(Np)]  # Control input bounds
Vmax=2.2  # Maximum volume (L)
Smin=0.1 # Keep substrate above this value (g/L)


#====================
# ADDING NOISE
#====================
biomass_noise_ratio = 0.05  # 5% noise
substrate_noise_ratio = 0.03  # 3% noise
noise="MEASURED" # Noise "MODEL" in model accuracy. We don't know the ground truth exactly./ Noise "MEASURED" in measured data. Only handled by the controller.

#====================
# SIMULATION CONFUGURATION  
#====================

# Calculate simulation steps
SS = int(At/dt)
np.random.seed(42)    # For reproducibility


# Parameter Estimation

In [5]:
def evaluate_tracking_performance(X, X_setpoint):
    error = X - X_setpoint
    mse = np.mean(error**2)
    
    overshoot = np.max(X) - X_setpoint
    overshoot_penalty = overshoot if overshoot > 0 else 0

    # Settling time: assume we want error < 5% of setpoint for last 5 steps
    tol = 0.01 * X_setpoint
    settling_penalty = np.sum(np.abs(error[-5:]) > tol)

    score = 10 * mse + 5 * overshoot_penalty + 20 * settling_penalty
    return -score

   

In [9]:
# ========================
# MPC MAIN LOOP
# ========================
def MPC_LOOP_OPT(constraint=False,Q_val=3310, R_val=151, Np_val=15, X_setpoint=30):
    global Q, R, Np
    Q, R, Np = Q_val, R_val, Np_val
    previous_F_opt = F0 * np.ones(Np)  # Initial guess for first optimization
    # Initialize arrays
    X = np.zeros(SS+1)
    S = np.zeros(SS+1)
    V = np.zeros(SS+1)
    F = np.zeros(SS)
    X[0], S[0], V[0] = X0, S0, V0
    bnds = [(0, 0.1) for _ in range(Np_val)]  # Dynamic bounds
   
    for step in range(SS):
        current_time = step * dt
        
        # Simulate measurement with noise
        X_measured = X[step] #+np.random.normal(0,X[step]*biomass_noise_ratio)
        S_measured = S[step] #+np.random.normal (0,S[step]*substrate_noise_ratio)
        F_guess =np.full(Np_val, previous_F_opt[-1])

        # Solve MPC optimization
        res = minimize(
                cost_function,
                F_guess,
                args=(X_measured, S_measured, V[step], current_time, Q_val, R_val, Np_val, h, dt),
                bounds=bnds,
                constraints={'type': 'ineq', 'fun': lambda F_opt: Volume_constraint(F_opt, X[step], S[step], V[step], current_time)} if constraint else None,
                method="SLSQP",
            )

        
        previous_F_opt = res.x
        # Apply first control input
        F[step] = res.x[0]
        
        # Simulate real system with actual model
        sol = solve_ivp(
            actual_model,
            [current_time, current_time + dt],
            [X[step], S[step], V[step]],
            args=(F[step],),
            t_eval=[current_time,current_time + dt],
        )
        X[step+1], S[step+1], V[step+1] = sol.y[:, -1]
    return evaluate_tracking_performance(X, X_setpoint)



In [10]:
def objective(trial):
    # Let Optuna suggest values
    Q_val = trial.suggest_float("Q", 1, 100,step=1)
    R_val = trial.suggest_float("R", 1,100, step=1)
    Np_val = trial.suggest_int("Np", 8,8)

    # Run your MPC controller with these parameters
    score = MPC_LOOP_OPT(
        constraint=False,
        Q_val=Q_val,
        R_val=R_val,
        Np_val=Np_val,
        X_setpoint=30
    )

    return -score  # because Optuna minimizes by default



# ========================
# OPTIMIZATION WITH CHANGING SEED
# ========================
# def objective(trial):
#     Q_val = trial.suggest_float("Q", 1, 1000, log=False, step=1)
#     R_val = trial.suggest_float("R", 7000, 10000, log=False, step=1)
#     Np_val = trial.suggest_int("Np",12,12)
    
#     scores = []
#     for seed in np.random.randint(0,1000,30):  # try 3 different random seeds
#         np.random.seed(seed)  # ensure variability
#         score = MPC_LOOP_OPT(
#             constraint=False,
#             Q_val=Q_val,
#             R_val=R_val,
#             Np_val=Np_val,
#             X_setpoint=30
#         )
#         scores.append(score)
#     return -np.mean(scores)  # minimize average loss


In [11]:
study = optuna.create_study(direction="minimize")  # minimize MSE
# Inject your known good trial
study.enqueue_trial({
    "Q": 3,
    "R": 9970,
    "Np": 8  # or whatever you're using
})
study.optimize(objective, n_trials=50, timeout=50000)  # try 50 configurations or stop after 10 mins

[I 2025-05-15 11:16:04,248] A new study created in memory with name: no-name-fa4fe670-e837-4c20-b379-7d1574250058


[I 2025-05-15 11:16:12,866] Trial 0 finished with value: 1767.678039574866 and parameters: {'Q': 3.0, 'R': 9970.0, 'Np': 8}. Best is trial 0 with value: 1767.678039574866.
[I 2025-05-15 11:16:14,719] Trial 1 finished with value: 1857.4438436412822 and parameters: {'Q': 78.0, 'R': 23.0, 'Np': 8}. Best is trial 0 with value: 1767.678039574866.
[I 2025-05-15 11:16:17,422] Trial 2 finished with value: 1798.641177532258 and parameters: {'Q': 39.0, 'R': 18.0, 'Np': 8}. Best is trial 0 with value: 1767.678039574866.
[I 2025-05-15 11:16:18,825] Trial 3 finished with value: 1953.9800073224737 and parameters: {'Q': 58.0, 'R': 65.0, 'Np': 8}. Best is trial 0 with value: 1767.678039574866.
[I 2025-05-15 11:16:20,541] Trial 4 finished with value: 1923.5742463113431 and parameters: {'Q': 60.0, 'R': 65.0, 'Np': 8}. Best is trial 0 with value: 1767.678039574866.
[I 2025-05-15 11:16:22,227] Trial 5 finished with value: 2026.0630319152197 and parameters: {'Q': 77.0, 'R': 54.0, 'Np': 8}. Best is trial 0 

In [12]:
# Print the best solution
print("Best trial:")
trial = study.best_trial
print(f"Q = {trial.params['Q']:.2f}")
print(f"R = {trial.params['R']:.4f}")
print(f"Np = {trial.params['Np']}")
print(f"MSE = {-trial.value:.4f}")


Best trial:
Q = 1.00
R = 90.0000
Np = 8
MSE = -1767.2456


In [13]:
vis.plot_optimization_history(study).show()
vis.plot_param_importances(study).show()